# Siamese Neural Network (SNN) Training for Org Similarity

In [ ]:

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


## Load Data

In [ ]:
# Load required libraries
import pandas as pd
import numpy as np
from features import load_and_prepare_data

# Load features and positive/negative pairs
features_df = pd.read_csv('org_features.csv')
pairs_df = pd.read_csv('org_pairs.csv')  # columns: org1_id, org2_id, label

# Prepare input tensors for training
X1, X2, y = load_and_prepare_data(features_df, pairs_df)
print(f"Prepared {len(y)} samples.")

## Define SNN Model

In [ ]:

class SiameseNetwork(nn.Module):
    def __init__(self, input_dim):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )
    def forward_once(self, x):
        return self.fc(x)
    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return torch.cosine_similarity(out1, out2)


## Train Model

In [ ]:

# Training loop (simplified)
input_dim = X1.shape[1]
model = SiameseNetwork(input_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

X1_tensor = torch.tensor(X1).float()
X2_tensor = torch.tensor(X2).float()
y_tensor = torch.tensor(y).float()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(X1_tensor, X2_tensor)
    loss = torch.mean((outputs - y_tensor)**2)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
